In [3]:
import pandas as pd
import numpy as np

path = ''

filename = 'Data_Analysis_Temp.xlsx'


In [4]:
data = pd.read_excel(path+filename, sheet_name='3')
data.shape
print(data.head(10))

SF_F          SF_p     SC_chi2           SC_p     LR_chi2  \
0  151.288614  3.070620e-32  151.795160   7.024180e-35  140.261457   
1   18.144873  2.291907e-05   18.213258   1.974993e-05   18.008773   
2   13.043168  3.262740e-04   13.099388   2.953920e-04   12.978158   
3  102.472420  2.475630e-22  102.988221   3.371470e-24   95.034361   
4   87.252722  3.226049e-19   87.780461   7.313788e-21   80.860281   
5  142.166945  3.243140e-28  143.243968   5.198630e-33  122.395089   
6  144.347447  2.369320e-27  145.810428   1.428190e-33  118.764179   
7  118.612983  6.645260e-22  120.428488   5.097150e-28   94.172491   
8  547.737748  1.883460e-41  564.854552  7.389200e-125  188.391379   
9  237.039198  1.246880e-47  237.832856   1.167480e-53  211.011881   

           LR_p        PF_F          PF_p  Itrctn    Note  
0  2.333670e-32  151.288614  3.070620e-32    True  1 -> 2  
1  2.198892e-05   18.144873  2.291907e-05    True  1 -> 2  
2  3.151460e-04   13.043168  3.262740e-04    True  1 -> 2 

In [7]:
sorted_data = data.sort_values(by=['SF_p'], ascending=True)
f_test_data = sorted_data[['SF_F', 'SF_p', 'Itrctn', 'Note']]
f_test_data.reset_index(drop=True, inplace=True)
total_test = len(f_test_data.index)
p_raw = 0.05

print(f_test_data.head(55))
print("Total entries:", total_test)

SF_F          SF_p  Itrctn    Note
0   257.260778  4.367490e-51    True  2 -> 3
1   237.039198  1.246880e-47    True  2 -> 1
2   547.737748  1.883460e-41    True  1 -> 2
3   183.807637  3.181510e-38    True  3 -> 2
4   443.560074  9.170050e-38    True  3 -> 2
5   194.051690  3.544220e-36    True  2 -> 1
6   218.170960  1.121990e-33    True  2 -> 3
7   151.288614  3.070620e-32    True  1 -> 2
8   152.177899  9.232150e-32    True  2 -> 1
9   147.576095  6.207520e-31    True  2 -> 3
10  184.472416  4.781640e-30    True  2 -> 1
11  138.462822  6.882740e-29    True  2 -> 3
12  137.490031  1.024500e-28    True  2 -> 1
13  142.166945  3.243140e-28    True  1 -> 2
14  144.347447  2.369320e-27    True  1 -> 2
15  125.518705  1.432600e-26    True  3 -> 2
16  131.570951  1.710750e-26    True  2 -> 3
17  189.406591  1.919710e-24    True  2 -> 3
18  123.474738  3.302010e-24    True  3 -> 2
19  123.388041  3.405410e-24    True  2 -> 1
20  131.858112  1.131670e-23    True  3 -> 2
21  177.113480  1.60

In [8]:
# Bonferroni Default:
def bonferroni_default(p):
    result = str(p*(1/total_test))
    return result

In [9]:
print(bonferroni_default(p_raw))

0.000925925925925926


In [10]:
# FDR by Benjamini and Hochberg
def fdr_bh(order, curr_p):
    result = []
    p_k_ref = p_raw*(order/total_test)
    q_value = curr_p*(total_test/order)
    result.append(p_k_ref)
    result.append(q_value)
    return result


In [11]:
p_ref_list = []
q_value_list = []
compare_1 = []
compare_2 = []
for i in f_test_data.index:
    p_value = f_test_data['SF_p'][i]
    p_ref = '%e' % (fdr_bh(i+1, p_value)[0])
    q_value = fdr_bh(i+1, p_value)[1]

    p_ref_list.append(p_ref)
    q_value_list.append(q_value)
    compare_1.append(float(p_value) < float(p_ref))
    compare_2.append(float(p_value) < float(q_value))

    #print("Rank:", i+1, p_value, p_ref, q_value)
for i in range(len(q_value_list)-1):
    if q_value_list[i+1] > q_value_list[i]:
        q_value_list[i] = q_value_list[i+1]

df_bh = pd.DataFrame(
    {"p_ref": p_ref_list,
     "q_value": q_value_list,
     "p<p_k": compare_1,
     "p<q": compare_2}
)

print(pd.concat([f_test_data['SF_p'], df_bh, f_test_data['Itrctn']], axis=1))


SF_p         p_ref       q_value  p<p_k    p<q  Itrctn
0   4.367490e-51  9.259259e-04  3.366576e-46   True   True    True
1   1.246880e-47  1.851852e-03  3.390228e-40   True   True    True
2   1.883460e-41  2.777778e-03  4.295038e-37   True   True    True
3   3.181510e-38  3.703704e-03  9.903654e-37   True   True    True
4   9.170050e-38  4.629630e-03  3.189798e-35   True   True    True
5   3.544220e-36  5.555556e-03  8.655351e-33   True   True    True
6   1.121990e-33  6.481481e-03  2.072668e-31   True   True    True
7   3.070620e-32  7.407407e-03  5.539290e-31   True   True    True
8   9.232150e-32  8.333333e-03  3.352061e-30   True   True    True
9   6.207520e-31  9.259259e-03  2.347351e-29   True   True    True
10  4.781640e-30  1.018519e-02  3.097233e-28   True   True    True
11  6.882740e-29  1.111111e-02  4.255615e-28   True   True    True
12  1.024500e-28  1.203704e-02  1.250925e-27   True   True    True
13  3.243140e-28  1.296296e-02  8.529552e-27   True   True    True
14  2.3